In [618]:
import pandas as pd
import numpy as np 
import re
import random
import copy

In [619]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet1")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet3")

In [620]:
df.head()

,QUESTION_LABEL,TARGET_QUESTION,INTENT_DESCRIPTION,EXAMPLE_SENTENCE,QUESTION_SHOULD_CONTAIN,Unnamed: 5
0,buy,Where can I buy a @partType for my @appliance?...,The sentence specifies that the user wants to ...,Where can I buy a glass tray for my microwave?...,@partType @appliance,NaN
1,clean,How do I clean the @cleanObject on the @applia...,The sentence asks how to clean a part of the a...,How do I clean the glass cooktop on the oven?\...,@cleanObject @appliance,NaN
2,code,What does @infoCode mean on my @appliance?\nMy...,The sentence is asking about the\nmeaning of t...,What is the c-d0 mean on my oven?\nMy microwav...,@infoCode @appliance,Information Code
3,appropriate,Is it safe to use @cookWare in a @appliance?\n...,A sentence that asks if it is safe to use a ce...,Is it safe to use Glassware in the microwave?\...,@cookWare @appliance,NaN
4,remove,How do I remove @removeObject from the @applia...,The sentence asks how to remove something\nfro...,How do I remove burned-on residue from the ove...,@removeObject @appliance,NaN


In [621]:
df['QUESTION_SHOULD_CONTAIN']= df['QUESTION_SHOULD_CONTAIN'].str.lower()
slot_df['Slot']= slot_df['Slot'].str.lower()
df['TARGET_QUESTION']= df['TARGET_QUESTION'].str.lower()
df["TARGET_QUESTION"] = df['TARGET_QUESTION'].str.replace('[\.\?\']','')

In [622]:
## Preprocessing slot_df dataframe
# Slot dataframe trimmed so that the Description is removed. 
slot_df_clean = slot_df.iloc[:,[0,2]]
# replace the Example column with an array of the values separated by a comma
slot_df_clean = slot_df_clean[slot_df_clean['Examples'].notnull()].copy()
slot_df_clean.loc[:,'Examples'] = slot_df_clean.loc[:,"Examples"].apply(lambda x: x.split(', '))
# dictionary of the slots. Key: slot value: possible vallues of the slot. 
slot_dict = dict(zip(slot_df_clean.Slot, slot_df_clean.Examples))

In [623]:
# Split the TARGET_QUESTION column and make it into an array of questions
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: x.split('\n'))
#Temporary extract the TARGET_QUESTION column that was processed above. 
target = df.iloc[:,1]

In [624]:
# A Series of arrays representing the respective slots
slots = df.iloc[:,4].apply(lambda x: x.split(' '))
# The dictionary where key:Intent value:Question
target_dic = df.iloc[:,[0,1]]
# A dictionary of intents key: int value: intent
intent_dict = df.QUESTION_LABEL.to_dict()

In [625]:
slot_dict_boi = copy.deepcopy(slot_dict)
for key,value in slot_dict.items():
    list_slot = []
    for i in value:
        s = i.split()
        for j in range(len(s)):
            if j == 0:
                s[j] = s[j] + ":B-" + key[1:]
            else:
                s[j] = s[j] + ":I-" + key[1:]
        i = ' '.join(s)
        list_slot.append(i)
    slot_dict_boi[key] = list_slot

In [626]:
template_dict = {}
edge_dict = {}
# template_dict['buy'] = 
# template_dict['clean'] = 
# template_dict['code'] = 
# template_dict['appropriate'] = 
# template_dict['remove'] = 
# template_dict['replace'] = 
# template_dict['cook'] = 
# template_dict['purpose'] = 
# template_dict['set'] = 
# template_dict['use']=
# template_dict['on_off'] = 
edge_dict['buy'] = 'buy parts for appliance'
edge_dict['clean'] = 'clean part of a appliance'
edge_dict['code'] = 'error code on appliance screen'
edge_dict['appropriate'] ='appropriate kitchen untensils to use'
edge_dict['remove'] = 'how to remove part from appliance'
edge_dict['replace'] = 'how to replace part from appliance'
edge_dict['cook'] = 'how to cook food on an appliance'
edge_dict['purpose'] = 'the purpose of an appliance feature'
edge_dict['set'] = 'how to set an appliance feature'
edge_dict['use'] = 'how to use an appliance feature'
edge_dict['on_off'] = 'how to turn on/off a feature on an appliance.'
edge_dict['image'] = 'show the image of a part or button in an appliance'
edge_dict['locate'] = 'locate a specific part or button on the appliance'
edge_dict['exist'] = 'Does a certain feature exist?'

In [627]:
edge_to_intent = dict((v, k) for k, v in edge_dict.items())

In [628]:
edge_to_intent

{'buy parts for appliance': 'buy',
 'clean part of a appliance': 'clean',
 'error code on appliance screen': 'code',
 'appropriate kitchen untensils to use': 'appropriate',
 'how to remove part from appliance': 'remove',
 'how to replace part from appliance': 'replace',
 'how to cook food on an appliance': 'cook',
 'the purpose of an appliance feature': 'purpose',
 'how to set an appliance feature': 'set',
 'how to use an appliance feature': 'use',
 'how to turn on/off a feature on an appliance.': 'on_off',
 'show the image of a part or button in an appliance': 'image',
 'locate a specific part or button on the appliance': 'locate',
 'Does a certain feature exist?': 'exist'}

In [629]:
output = {"_worker_id": [], "edge": [], "template": [], "question_should_contain": [], "question": [], "intent":[]}
i = 0
for s in range(len(target)): #Intent number
    print("Intent:",intent_dict[s],"|","number of slots:",len(slots[s]))
    for j in range(len(target[s])): #Sentence in the intent
        print(target[s][j])
        print(slots[s])
        print(' '.join(slots[s]))
        output["_worker_id"].append("-")
        output["edge"].append(edge_dict[intent_dict[s]])
        output["template"].append("-")
        output["question_should_contain"].append(' '.join(slots[s]))
        output["question"].append(target[s][j])
        output["intent"].append(intent_dict[s])
        print(i)
        i+=1

Intent: buy | number of slots: 2
where can i buy a @parttype for my @appliance
['@parttype', '@appliance']
@parttype @appliance
0
i want to get a new @parttype for my @appliance
['@parttype', '@appliance']
@parttype @appliance
1
how can i purchase a @parttype for my @appliance
['@parttype', '@appliance']
@parttype @appliance
2
where can i find a new @parttype for my @appliance
['@parttype', '@appliance']
@parttype @appliance
3
Intent: clean | number of slots: 2
how do i clean the @cleanobject on the @appliance
['@cleanobject', '@appliance']
@cleanobject @appliance
4
tell me how to polish the @cleanobject with the @appliance
['@cleanobject', '@appliance']
@cleanobject @appliance
5
what is the best way to clean the @cleanobject on the @appliance
['@cleanobject', '@appliance']
@cleanobject @appliance
6
i would like to clean the @cleanobject with the @appliance please
['@cleanobject', '@appliance']
@cleanobject @appliance
7
Intent: code | number of slots: 2
what does @infocode mean on my @

In [630]:
df_pandas = pd.DataFrame.from_dict(output)
df_pandas

,_worker_id,edge,template,question_should_contain,question,intent
0,-,buy parts for appliance,-,@parttype @appliance,where can i buy a @parttype for my @appliance,buy
1,-,buy parts for appliance,-,@parttype @appliance,i want to get a new @parttype for my @appliance,buy
2,-,buy parts for appliance,-,@parttype @appliance,how can i purchase a @parttype for my @appliance,buy
3,-,buy parts for appliance,-,@parttype @appliance,where can i find a new @parttype for my @appli...,buy
4,-,clean part of a appliance,-,@cleanobject @appliance,how do i clean the @cleanobject on the @appliance,clean
5,-,clean part of a appliance,-,@cleanobject @appliance,tell me how to polish the @cleanobject with th...,clean
6,-,clean part of a appliance,-,@cleanobject @appliance,what is the best way to clean the @cleanobject...,clean
7,-,clean part of a appliance,-,@cleanobject @appliance,i would like to clean the @cleanobject with th...,clean
8,-,error code on appliance screen,-,@infocode @appliance,what does @infocode mean on my @appliance,code
9,-,error code on appliance screen,-,@infocode @appliance,my @appliance has the @infocode message,code


In [631]:
print(len(output['_worker_id']))
print(len(output['edge']))
print(len(output['template']))
print(len(output['question_should_contain']))
print(len(output['question']))
print(len(output['intent']))

44
44
44
44
44
44


In [632]:
df = pd.read_excel("appliance_utterances_test.xlsx")

In [633]:
df_trim = df.iloc[:,2:]

In [634]:
df_trim['intent'] = df_trim['edge'].apply(lambda row : edge_to_intent[row])

In [635]:
df_trim.loc[24,'question']

'Is @cookWare a high quality appliance'

In [636]:
df_trim.drop([24,25,26,98,99,184,226],axis=0,inplace=True)

In [637]:
df_ = pd.concat([df_trim, df_pandas], axis=0)
df_.head()

,_worker_id,edge,template,question_should_contain,question,intent
0,43844929,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Do you know if @appliance has an @existFeature...,exist
1,43844929,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Can you tell me if my @appliance has an @exist...,exist
2,43844929,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Is there an @existFeature on my new @appliance,exist
3,38202325,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Is the @existFeature available on the @appliance,exist
4,38202325,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Does the @appliance offer an @exist Feature,exist


In [638]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281 entries, 0 to 43
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   _worker_id               281 non-null    object
 1   edge                     281 non-null    object
 2   template                 281 non-null    object
 3   question_should_contain  281 non-null    object
 4   question                 281 non-null    object
 5   intent                   281 non-null    object
dtypes: object(6)
memory usage: 15.4+ KB


In [639]:
df_ = df_.sort_values(by=['edge'])
df_.reset_index(drop=True,inplace=True)

In [640]:
file_name = 'appliance_utterances_appen_personal.xlsx'

In [641]:
df_.to_excel(f"{file_name.replace('.xlsx', '')}.xlsx")
df_.to_csv(f"{file_name.replace('.xlsx', '')}.tsv", sep="\t")

In [642]:
data = pd.read_excel(file_name)

In [643]:
data

,Unnamed: 0,_worker_id,edge,template,question_should_contain,question,intent
0,0,43844929,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Do you know if @appliance has an @existFeature...,exist
1,1,45405037,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Will there be an @existFeature in the @appliance,exist
2,2,45405037,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Can you find me an @existFeature in the @appli...,exist
3,3,45405037,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Can I see an @existFeature in the @appliance,exist
4,4,38202325,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,Does the @appliance offer the @existFeature,exist
...,...,...,...,...,...,...,...
276,276,43844929,the purpose of an appliance feature,What is the purpose of the @purposeFeature on ...,@purposeFeature @appliance,Can you tell me what the purposes of @purposeF...,purpose
277,277,43844929,the purpose of an appliance feature,What is the purpose of the @purposeFeature on ...,@purposeFeature @appliance,"What, exactly, is the purpose of @purposeFeatu...",purpose
278,278,18090380,the purpose of an appliance feature,What is the purpose of the @purposeFeature on ...,@purposeFeature @appliance,How do I use the @purposeFeature function on m...,purpose
279,279,-,the purpose of an appliance feature,-,@purposefeature @appliance,what can i use the @purposefeature on the @app...,purpose


In [644]:
data_trim = data[(data.intent !='exist') & (data.intent != 'locate') & (data.intent != 'image')]

In [645]:
data_trim.head()

,Unnamed: 0,_worker_id,edge,template,question_should_contain,question,intent
18,18,6367365,appropriate kitchen untensils to use,Is it safe to use @cookWare in a @appliance?,@cookWare @appliance,Will I be able to use @cookWare in the @applia...,appropriate
19,19,-,appropriate kitchen untensils to use,-,@cookware @appliance,can i use the @cookware for the @appliance,appropriate
20,20,-,appropriate kitchen untensils to use,-,@cookware @appliance,is @cookware appropriate for the @appliance,appropriate
21,21,-,appropriate kitchen untensils to use,-,@cookware @appliance,is the @cookware @appliance safe,appropriate
22,22,-,appropriate kitchen untensils to use,-,@cookware @appliance,is it safe to use @cookware in a @appliance,appropriate


In [646]:
data_trim["question"] = data_trim['question'].str.replace('[\.\?\'\,]','')

C:\Users\l.kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [647]:
data_trim['question']= data_trim['question'].str.lower()
data_trim['question_should_contain']= data_trim['question_should_contain'].str.lower()

C:\Users\l.kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\l.kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [648]:
data_trim.reset_index(drop=True,inplace=True)

In [649]:
data_trim.loc[0,'intent'] == 'exist'

False

In [650]:
data_trim.loc[0,'question']

'will i be able to use @cookware in the @appliance '

In [651]:
data_trim.loc[28,'question']

'where can i purchase @parttype  from my @appliance'

In [652]:
data_trim.shape[0]

227

In [653]:
a = data_trim.loc[0,'question_should_contain'].split(' ')
a[0]

'@cookware'

In [654]:
slot_dict

{'@appliance': ['microwavawe', 'oven'],
 '@parttype': ['door',
  'light',
  'wirerack',
  'air fry tray',
  'storage drawer',
  'graphite lubricant',
  'glass tray',
  'roller guide ring',
  'exhaust filter',
  'grease filter',
  'charcoal filter',
  'hardware kit',
  'screws',
  'brackets'],
 '@cleanobject': ['grease filter',
  'cooktop',
  'cooktop seal',
  'door gasket',
  'stainless steel',
  'glass cooktop',
  'sugary-spills',
  'melted-plastic'],
 '@infocode': ['c-d0',
  'c-d1',
  'c-F0',
  'c-20',
  'c-21',
  'c-22',
  'c-30',
  'c-31',
  'c-A2',
  'bAd-LInE'],
 '@cookware': ['aluminum foil',
  'browning plate',
  'ceramic',
  'porcelain',
  'stoneware',
  'polyester cardboard',
  'polyester cup',
  'paper bags',
  'recycled paper',
  'metal trimes',
  'glassware',
  'metal',
  'paper',
  'plastic',
  'thermometer',
  'straw',
  'wicker',
  'wood'],
 '@removeobject': ['rack', 'residue', 'stains', 'door', 'drawer', 'lights'],
 '@replaceobject': ['rack',
  'door',
  'drawer',
  'l

In [655]:
slot_dict_boi

{'@appliance': ['microwavawe:B-appliance', 'oven:B-appliance'],
 '@parttype': ['door:B-parttype',
  'light:B-parttype',
  'wirerack:B-parttype',
  'air:B-parttype fry:I-parttype tray:I-parttype',
  'storage:B-parttype drawer:I-parttype',
  'graphite:B-parttype lubricant:I-parttype',
  'glass:B-parttype tray:I-parttype',
  'roller:B-parttype guide:I-parttype ring:I-parttype',
  'exhaust:B-parttype filter:I-parttype',
  'grease:B-parttype filter:I-parttype',
  'charcoal:B-parttype filter:I-parttype',
  'hardware:B-parttype kit:I-parttype',
  'screws:B-parttype',
  'brackets:B-parttype'],
 '@cleanobject': ['grease:B-cleanobject filter:I-cleanobject',
  'cooktop:B-cleanobject',
  'cooktop:B-cleanobject seal:I-cleanobject',
  'door:B-cleanobject gasket:I-cleanobject',
  'stainless:B-cleanobject steel:I-cleanobject',
  'glass:B-cleanobject cooktop:I-cleanobject',
  'sugary-spills:B-cleanobject',
  'melted-plastic:B-cleanobject'],
 '@infocode': ['c-d0:B-infocode',
  'c-d1:B-infocode',
  'c-F0

In [656]:
slot_dict[a[0]]

['aluminum foil',
 'browning plate',
 'ceramic',
 'porcelain',
 'stoneware',
 'polyester cardboard',
 'polyester cup',
 'paper bags',
 'recycled paper',
 'metal trimes',
 'glassware',
 'metal',
 'paper',
 'plastic',
 'thermometer',
 'straw',
 'wicker',
 'wood']

In [657]:
def random_slot(slot):
    random_slot = random.randint(0, len(slot_dict[slot])-1)
#     print(slot_dict[slot])
#     print(slot_dict[slot][random_slot])
    return slot_dict_boi[slot][random_slot]

In [658]:
slots = data_trim.loc[0,'question_should_contain'].split(' ')
slots

['@cookware', '@appliance']

In [659]:
random_slot(slots[0])

'polyester:B-cookware cardboard:I-cookware'

In [660]:
f_train = open('appliance2\\train','w')
f_test = open('appliance2\\test','w')
f_valid = open('appliance2\\valid','w')
with open('appliance2\\train','w') as f_train, open('appliance2\\test','w') as f_test, open('appliance2\\valid','w') as f_valid:
    for i in range(data_trim.shape[0]):
    #    if(i < 30):
    #         print(data_trim.loc[i,'question'])
    #         print(data_trim.loc[i,'question_should_contain'].split(' '))
        for j in range(5):
            slots = data_trim.loc[i,'question_should_contain'].split(' ')
            x = data_trim.loc[i,'question']
            x = re.sub(slots[0], random_slot(slots[0]), x)
            x = re.sub(slots[1], random_slot(slots[1]), x)
            x = x.lstrip()
            x = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            #print(data_trim.loc[i,'intent'])
            x = ' '.join(x.split())
            if((i % 4 == 0) or (i % 4 == 1)):
                f_train.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
                print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
            elif (i % 4 == 2):
                f_test.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
                print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
            else:
                f_valid.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
                print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
f_train.close()
f_test.close()
f_valid.close()

will:O i:O be:O able:O to:O use:O aluminum:B-cookware foil:I-cookware in:O the:O oven:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O straw:B-cookware in:O the:O microwavawe:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O straw:B-cookware in:O the:O microwavawe:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O porcelain:B-cookware in:O the:O microwavawe:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O paper:B-cookware bags:I-cookware in:O the:O oven:B-appliance <=> appropriate

can:O i:O use:O the:O wicker:B-cookware for:O the:O microwavawe:B-appliance <=> appropriate

can:O i:O use:O the:O wicker:B-cookware for:O the:O oven:B-appliance <=> appropriate

can:O i:O use:O the:O stoneware:B-cookware for:O the:O oven:B-appliance <=> appropriate

can:O i:O use:O the:O stoneware:B-cookware for:O the:O oven:B-appliance <=> appropriate

can:O i:O use:O the:O paper:B-cookware bags:I-cookware for:O the:O microwavawe:B-appliance <=> appropri


how:O work:O night:B-replaceobject light:I-replaceobject for:O the:O oven:B-appliance <=> replace

how:O work:O cooktop:B-replaceobject for:O the:O oven:B-appliance <=> replace

how:O work:O drawer:B-replaceobject for:O the:O oven:B-appliance <=> replace

how:O work:O charcoal:B-replaceobject filter:I-replaceobject for:O the:O microwavawe:B-appliance <=> replace

how:O work:O oven:B-replaceobject light:I-replaceobject for:O the:O microwavawe:B-appliance <=> replace

show:O me:O how:O to:O replace:O the:O night:B-replaceobject light:I-replaceobject on:O the:O oven:B-appliance <=> replace

show:O me:O how:O to:O replace:O the:O night:B-replaceobject light:I-replaceobject on:O the:O microwavawe:B-appliance <=> replace

show:O me:O how:O to:O replace:O the:O oven:B-replaceobject light:I-replaceobject on:O the:O microwavawe:B-appliance <=> replace

show:O me:O how:O to:O replace:O the:O lights:B-replaceobject on:O the:O oven:B-appliance <=> replace

show:O me:O how:O to:O replace:O the:O d

what:O does:O the:O cook:B-purposefeature time:I-purposefeature found:O on:O the:O microwavawe:B-appliance do:O <=> purpose

what:O does:O the:O timer:B-purposefeature found:O on:O the:O oven:B-appliance do:O <=> purpose

what:O does:O the:O bake:B-purposefeature found:O on:O the:O microwavawe:B-appliance do:O <=> purpose

what:O does:O the:O smart:B-purposefeature control:I-purposefeature found:O on:O the:O oven:B-appliance do:O <=> purpose

what:O does:O the:O steam:B-purposefeature clean:I-purposefeature found:O on:O the:O microwavawe:B-appliance do:O <=> purpose

can:O you:O tell:O me:O what:O the:O purposes:O of:O number:B-purposefeature pad:I-purposefeature are:O on:O this:O microwavawe:B-appliance <=> purpose

can:O you:O tell:O me:O what:O the:O purposes:O of:O keep:B-purposefeature warm:I-purposefeature are:O on:O this:O microwavawe:B-appliance <=> purpose

can:O you:O tell:O me:O what:O the:O purposes:O of:O setting:B-purposefeature are:O on:O this:O oven:B-appliance <=> purp

In [661]:
# Create appliance vocabulary for slots. 

slot_array = [] 
for key,value in slot_dict_boi.items():
    for i in value:
        slot_array = slot_array + i.split()
slot_array.sort()
slot_vocab = []
for i in slot_array:
    slot_vocab.append(i.split(':',1)[1])
    
slot_vocab.sort()
slot_vocab = list(dict.fromkeys(slot_vocab))

# for i in slot_vocab:
#     print(i)
    
f = open('appliance2\\vocab.slot','w')
for value in slot_vocab:
    f.write(value + "\n")
f.write("O")
f.close()

# Create appliance vocabulary for intents

f = open('appliance2\\vocab.intent','w')
for key,value in intent_dict.items():
    f.write(value + "\n")
f.close()

In [662]:
def boi_save(intent_number = len(target), single = False):
    f_train = open('appliance2\\train','w')
    f_test = open('appliance2\\test','w')
    f_valid = open('appliance2\\valid','w')
    with open('appliance2\\train','w') as f_train, open('appliance2\\test','w') as f_test, open('appliance2\\valid','w') as f_valid:
        line = ""
        start = 0
        end = intent_number
        if (single == True):
            start = intent_number - 1
            end = intent_number
        for s in range(start,end): #Intent number
    #         print("-----------------------------------------------")
            print(target[s])
            print("Intent:",intent_dict[s],"|","number of slots:",len(slots[s]))
            for j in range(len(target[s])): #Sentence in the intent
                if (len(slots[s]) == 2):
                    line,_ = boi_two_slots_write(s,j,f_train,f_test,f_valid)    
    #     print("-----------------------------------------------")
        f_train.close()
        f_test.close()
        f_valid.close()
        
def boi_two_slots_write(s,j,f_train,f_test,f_valid):
    p = 0
    x_boi = ""
    for i in range(len(slot_dict_boi[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict_boi[slots[s][1]])):#Slot 2
            x_original = target[s][j]
            x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x_original)
            x = re.sub(slots[s][1], slot_dict_boi[slots[s][1]][k], x)
            x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            p += 1
            #print(x_boi)
            if((j % 4 == 0) or (j % 4 == 1)):
                f_train.write(x_boi + " <=> " + intent_dict[s] + "\n")
            elif (j % 4 == 2):
                f_test.write(x_boi + " <=> " + intent_dict[s] + "\n")
            else:
                f_valid.write(x_boi + " <=> " + intent_dict[s] + "\n")

    return x, p